In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init('/home/ola/spark-3.0.3')

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import urllib
import pandas as pd
import wget

In [4]:
spark = SparkSession.builder.appName ("oladata").getOrCreate()

22/04/04 16:42:09 WARN Utils: Your hostname, ola-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.181.130 instead (on interface ens33)
22/04/04 16:42:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/04 16:42:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
import tarfile

In [6]:
# df= urllib.request.urlretrieve("https://cdn.captifytechnologies.com/samples/sampled_kws.tar.gz", "sampled_kws.tar.gz")

In [7]:
# tf = tarfile.open("sampled_kws.tar.gz")

In [8]:
# tf.extractall('./zip_file')

In [9]:
df1 =spark.read.format("parquet").load("./zip_file/sampled_kws/day=20220222")

In [10]:
df2 = spark.read.format("parquet").load("./zip_file/sampled_kws/day=20220223")

In [11]:
df3 = spark.read.format("parquet").load("./zip_file/sampled_kws/day=20220224")

In [12]:
from functools import reduce
from pyspark.sql import DataFrame

In [13]:
df_total =[df1,df2,df3]

In [14]:
#merging all data df1, df2 and df3 together because they
union_df = reduce(DataFrame.unionAll,df_total)

In [15]:
union_df.show()

+-------------------+--------------------+---------------+-----------+----------+--------------------+--------------------+----------+
|          eventDate|           keyphrase|      urlDomain|publisherId|regionCode|           userAgent|    hashed_concat_id|   anon_ip|
+-------------------+--------------------+---------------+-----------+----------+--------------------+--------------------+----------+
|2022-02-22 01:12:28|miley cyrus unzip...|  inquisitr.com|      12441|          |Mozilla/5.0 (Linu...|d1f758694c32be658...|2147483647|
|2022-02-22 01:21:54| hawaiian last names|               |      12437|     US-FL|Mozilla/5.0 (Linu...|23f340c7e217dc122...|1234471680|
|2022-02-22 01:02:29|lifestyle these p...|moneyawaits.com|      12441|          |Mozilla/5.0 (Linu...|4f8f018a71b986fff...| 643723776|
|2022-02-22 01:48:33|news costco disco...|    eatthis.com|      12441|          |Mozilla/5.0 (Linu...|1f33a4cfee82f158d...|1074392832|
|2022-02-22 01:18:54|general news - di...|             

In [16]:
#checking dataframe schema
union_df.printSchema()

root
 |-- eventDate: string (nullable = true)
 |-- keyphrase: string (nullable = true)
 |-- urlDomain: string (nullable = true)
 |-- publisherId: string (nullable = true)
 |-- regionCode: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- hashed_concat_id: string (nullable = true)
 |-- anon_ip: integer (nullable = true)



In [17]:
import sys
from operator import add
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
import pyspark.sql.functions as f

In [18]:
#spliting into keywords and counting
sf=union_df.withColumn('word', f.explode(f.split(f.col('keyphrase'), ' '))).groupBy('word').count().sort('count', ascending=False)

In [19]:
sf.show()

+-----+-------+
| word|  count|
+-----+-------+
|     |2364192|
| news|2308584|
|   de|1799190|
|    a|1711962|
|  and|1620867|
|    -|1539807|
|  the|1463751|
|   in|1289241|
|   to|1240044|
|   en|1094955|
|   la|1010274|
|   of| 968508|
|    e| 677859|
|  for| 660309|
|video| 506994|
|    i| 498261|
|   on| 492057|
|   tv| 476487|
|   le| 464058|
|   is| 462480|
+-----+-------+
only showing top 20 rows



In [20]:
#converts the input string to lowercase and splits it by white spaces
tokenizer = Tokenizer(inputCol="keyphrase", outputCol="words_token")

In [21]:
#apply transform
tokenized = tokenizer.transform(union_df).select('hashed_concat_id','words_token')

In [22]:
#show tokenized
tokenized.show()

+--------------------+--------------------+
|    hashed_concat_id|         words_token|
+--------------------+--------------------+
|d1f758694c32be658...|[miley, cyrus, un...|
|23f340c7e217dc122...|[hawaiian, last, ...|
|4f8f018a71b986fff...|[lifestyle, these...|
|1f33a4cfee82f158d...|[news, costco, di...|
|bfaa2a17d8d6c8f72...|[general, news, -...|
|352309d288ed17de4...|[brie, larson, se...|
|b54eb9076bc8dbec1...|[spring, data, re...|
|9962edbc375afb528...|       [main, frame]|
|e73cf0d6b78d0b09b...|[augmented, true,...|
|ffcab12b5f4f277b0...|[coast, guard, su...|
|d4632d6454f987fb5...|[bold, beautiful,...|
|cc5c79727479f2306...|[us, figure, skat...|
|f54908750e47c7618...|[hogs, haven, moc...|
|9e619420b6abd447f...|[dave, chappelle,...|
|f379eb1cbc67cf0b2...|[crown, royal, bu...|
|70549c0f0fc22a595...|[movies, indiana,...|
|0ca693631f439a073...|  [future, archives]|
|7c825e2990a139544...|[the, legend, of,...|
|fc62495d829cd0e06...|[7, new, movies, ...|
|7cccbdf540651e1b2...|[afc, west

In [23]:
#filters out stop words from words_token
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')

In [24]:
#apply transform
data_clean = remover.transform(tokenized).select('hashed_concat_id', 'words_clean')

In [25]:
data_clean.show()

+--------------------+--------------------+
|    hashed_concat_id|         words_clean|
+--------------------+--------------------+
|d1f758694c32be658...|[miley, cyrus, un...|
|23f340c7e217dc122...|[hawaiian, last, ...|
|4f8f018a71b986fff...|[lifestyle, peopl...|
|1f33a4cfee82f158d...|[news, costco, di...|
|bfaa2a17d8d6c8f72...|[general, news, -...|
|352309d288ed17de4...|[brie, larson, se...|
|b54eb9076bc8dbec1...|[spring, data, re...|
|9962edbc375afb528...|       [main, frame]|
|e73cf0d6b78d0b09b...|[augmented, true,...|
|ffcab12b5f4f277b0...|[coast, guard, su...|
|d4632d6454f987fb5...|[bold, beautiful,...|
|cc5c79727479f2306...|[us, figure, skat...|
|f54908750e47c7618...|[hogs, haven, moc...|
|9e619420b6abd447f...|[dave, chappelle,...|
|f379eb1cbc67cf0b2...|[crown, royal, bu...|
|70549c0f0fc22a595...|[movies, indiana,...|
|0ca693631f439a073...|  [future, archives]|
|7c825e2990a139544...|[legend, heroes, ...|
|fc62495d829cd0e06...|[7, new, movies, ...|
|7cccbdf540651e1b2...|[afc, west

In [26]:
result = data_clean.withColumn('word', f.explode(f.col('words_clean'))) \
  .groupBy('word') \
  .count().sort('count', ascending=False)

In [27]:
result.show()

+------+-------+
|  word|  count|
+------+-------+
|      |2364192|
|  news|2355078|
|    de|1801179|
|     -|1539807|
|    en|1095573|
|    la|1013835|
|     e| 679734|
|    tv| 515205|
| video| 509676|
|    le| 465087|
|    es| 443616|
|    un| 402609|
|    di| 387084|
|    us| 384423|
|    il| 353142|
|futbol| 346983|
|   que| 332037|
|  best| 324978|
|   les| 307719|
|   new| 302583|
+------+-------+
only showing top 20 rows



In [28]:
import pyspark.sql.functions as F

In [29]:
#create column for length of words
ff = result.withColumn("length_of_word", F.length("word"))

In [30]:
#show length of words
ff.show(truncate=False)

+------+-------+--------------+
|word  |count  |length_of_word|
+------+-------+--------------+
|      |2364192|0             |
|news  |2355078|4             |
|de    |1801179|2             |
|-     |1539807|1             |
|en    |1095573|2             |
|la    |1013835|2             |
|e     |679734 |1             |
|tv    |515205 |2             |
|video |509676 |5             |
|le    |465087 |2             |
|es    |443616 |2             |
|un    |402609 |2             |
|di    |387084 |2             |
|us    |384423 |2             |
|il    |353142 |2             |
|futbol|346983 |6             |
|que   |332037 |3             |
|best  |324978 |4             |
|les   |307719 |3             |
|new   |302583 |3             |
+------+-------+--------------+
only showing top 20 rows



In [31]:
#find when length of word is not zero
ff=ff.filter(ff.length_of_word!=0)

In [32]:
list_df = ff.select("word", "count").rdd.flatMap(lambda x: x).collect()

In [34]:
from pyspark.sql.functions import col

In [35]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
pandas_df = result.toPandas()
pandas_df.sort_values(by='count',ascending=False).plot(x ='word', y='count', kind = 'bar')

In [ ]:
#data_clean.filter(F.col('words_clean').isin(list_ddd)).show()